#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

e:\Gen AI\Langchain\1-Basics+Of+Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='\n\n\n\n\n🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2026 LangChain, Inc.\n\n')]

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2026 LangChain, Inc.')]

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

# Uses your local Ollama server (default: http://localhost:11434)
# Make sure you have an embedding model pulled, e.g.: `ollama pull nomic-embed-text`
embeddings = OllamaEmbeddings(model="nomic-embed-text")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18668\860198713.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


In [8]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [9]:
vectorstoredb

In [10]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2026 LangChain, Inc.'

In [11]:
from langchain_community.chat_models import ChatOllama

# Uses your local Ollama server (default: http://localhost:11434)
llm = ChatOllama(model="gemma:2b", temperature=0)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18668\1964310153.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma:2b", temperature=0)


In [12]:
## Retrieval Chain, Document chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
<context>
{context}
</context>

Question: {input}
"""
 )

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'), additional_kwargs={})])
| ChatOllama(model='gemma:2b', temperature=0.0)
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [13]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'Sure, based on the context, LangSmith has two usage limits: total traces and extended traces.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [14]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [15]:
retriever = vectorstoredb.as_retriever()

from langchain_classic.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000186D4024440>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Answer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'), addit

In [17]:
## Get the response from the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'The context does not provide information about the total traces and extended usage limits of LangSmith, so I cannot answer this question from the provided context.'

In [18]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='6c25f1c5-f98f-40e8-b85f-6b366bbf828f', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2026 LangChain, Inc.')],
 'answer': 'The context does not provide information about the total traces and extended usage limits of LangSmith, so I cannot answer this question from the provided context.'}

In [19]:
response['context']

[Document(id='6c25f1c5-f98f-40e8-b85f-6b366bbf828f', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentLangSmith docs have moved! Find the LangSmith docs at the new LangChain Docs site.API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityLangChain ForumTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2026 LangChain, Inc.')]

In [20]:
import importlib
import langchain

print('langchain:', getattr(langchain, '__version__', '(no __version__)'))
print('langchain file:', getattr(langchain, '__file__', None))

try:
    import langchain.chains  # noqa: F401
    print('langchain.chains: OK')
except Exception as e:
    print('langchain.chains: FAIL ->', type(e).__name__, e)

for mod in [
    'langchain.chains.combine_documents',
    'langchain.chains.combine_documents.stuff',
    'langchain_classic.chains.combine_documents',
    'langchain_classic.chains.combine_documents.stuff',
]:
    try:
        m = importlib.import_module(mod)
        print(mod, 'import: OK; create_stuff_documents_chain:', hasattr(m, 'create_stuff_documents_chain'))
    except Exception as e:
        print(mod, 'import: FAIL ->', type(e).__name__, e)

langchain: 1.2.6
langchain file: e:\Gen AI\Langchain\1-Basics+Of+Langchain\venv\Lib\site-packages\langchain\__init__.py
langchain.chains: FAIL -> ModuleNotFoundError No module named 'langchain.chains'
langchain.chains.combine_documents import: FAIL -> ModuleNotFoundError No module named 'langchain.chains'
langchain.chains.combine_documents.stuff import: FAIL -> ModuleNotFoundError No module named 'langchain.chains'
langchain_classic.chains.combine_documents import: OK; create_stuff_documents_chain: True
langchain_classic.chains.combine_documents.stuff import: OK; create_stuff_documents_chain: True
